# Purpose

### 2022-02-14
In this notebook I'll select the clusters for the new FPR experiments for Canada, UK, Australia, & India.

Note that this is supposed to be an SFW experiment, so we'll need to filter out subreddits that are `over_18` or rated as `X`.

In one sheet include BOTH subreddit names & subreddit IDs.

---

TODO: Haven't included place logic (e.g., add direction to: city, state, country subreddits.)


### Updates
2022-02-16: 
I [David] will update the QA sheet so that we have standardized columns/format. Otherwise it'll be more work for us to wait for country managers to format things and then standardize them after the fact



# Imports & notebook setup

In [ ]:
%load_ext autoreload
%autoreload 2

# Register bigquery magic (only needed for laptop/local, not colab)
# %load_ext google.cloud.bigquery

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# colab auth for BigQuery
from google.colab import auth, files, drive
auth.authenticate_user()
print('Authenticated')

Authenticated


## Install custom library

### Append google drive path so we can install library from there

In [ ]:
# Attach google drive & import my python utility functions
# if drive.mount() fails, you can also:
#   MANUALLY CLICK ON "Mount Drive"
import sys


g_drive_root = '/content/drive'

try:
    drive.mount(g_drive_root, force_remount=True)
    print('   Authenticated & mounted Google Drive')
    
except Exception as e:
    try:
        drive._mount(g_drive_root, force_remount=True)
        print('   Authenticated & mounted Google Drive')
    except Exception as e:
        print(e)
        raise Exception('You might need to manually mount google drive to colab')

l_paths_to_append = [
    f'{g_drive_root}/MyDrive/Colab Notebooks',

    # need to append the path to subclu so that colab can import things properly
    f'{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n'
]
for path_ in l_paths_to_append:
    if path_ in sys.path:
        sys.path.remove(path_)
    print(f" Appending path: {path_}")
    sys.path.append(path_)

Mounted at /content/drive
   Authenticated & mounted Google Drive
 Appending path: /content/drive/MyDrive/Colab Notebooks
 Appending path: /content/drive/MyDrive/Colab Notebooks/subreddit_clustering_i18n


### Install library

In [ ]:
# install subclu & libraries needed to read parquet files from GCS & spreadsheets
#  make sure to use the [colab] `extra` because it includes colab-specific libraries
module_path = f"{g_drive_root}/MyDrive/Colab Notebooks/subreddit_clustering_i18n/[colab]"

!pip install -e $"$module_path" --quiet

     |████████████████████████████████| 10.1 MB 5.4 MB/s 
     |████████████████████████████████| 14.2 MB 36.5 MB/s 
     |████████████████████████████████| 965 kB 63.5 MB/s 
     |████████████████████████████████| 144 kB 60.2 MB/s 
     |████████████████████████████████| 285 kB 52.3 MB/s 
     |████████████████████████████████| 13.2 MB 36.2 MB/s 
     |████████████████████████████████| 79.9 MB 106 kB/s 
     |████████████████████████████████| 133 kB 54.7 MB/s 
     |████████████████████████████████| 715 kB 66.5 MB/s 
     |████████████████████████████████| 74 kB 3.0 MB/s 
     |████████████████████████████████| 112 kB 50.6 MB/s 
     |████████████████████████████████| 180 kB 50.7 MB/s 
     |████████████████████████████████| 79 kB 7.3 MB/s 
     |████████████████████████████████| 146 kB 69.4 MB/s 
     |████████████████████████████████| 58 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 65.4 MB/s 
     |████████████████████████████████| 596 kB 53.2 MB/s 
     |████████████

## Regular Imports

In [ ]:
import os
from datetime import datetime

from google.cloud import bigquery

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib_venn import venn2_unweighted, venn3_unweighted
from tqdm import tqdm


# auth for google sheets
import gspread
from oauth2client.client import GoogleCredentials


gc = gspread.authorize(GoogleCredentials.get_application_default())

# os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-science-prod-218515'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'

## Custom imports

In [ ]:
# subclu imports
import subclu
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric, reorder_array,
)
from subclu.models.clustering_utils import (
    create_dynamic_clusters,
    convert_distance_or_ab_to_list_for_fpr,
    reshape_df_to_get_1_cluster_per_row,
    get_primary_topic_mix_cols,
)

from subclu.models.reshape_clusters_v041 import (
    keep_only_target_labels,
    get_table_for_optimal_dynamic_cluster_params,
    get_dynamic_cluster_summary,
)


setup_logging()
notebook_display_config()
print_lib_versions([gspread, pd, np])

python		v 3.7.12
===
gspread		v: 4.0.1
pandas		v: 1.3.5
numpy		v: 1.21.5


# Checklist to re-run for a country:

- change country name in geo-relevance query
- change country name for google sheet name
- change country initial in google sheet
    - add google sheet KEY (after created)

# Load data from BigQuery

## Load subreddit geo-relevance & cultural relevance metadata

This data is already in bigQuery so read it straight from there. We'll use it to filter out geo-relevant (German) subs.

Also add the latest ratings so that we can filter based on those.

English-speaking countries don't have ambassador subs right now, so we should be able to create a standard template and replace the country name for these queries.

### SQL geo & cultural

In [ ]:
%%time
%%bigquery df_geo_and_lang --project data-science-prod-218515 

-- Select geo+cultural subreddits for a target country
--  And add latest rating & over_18 flags to exclude X-rated & over_18
DECLARE TARGET_COUNTRY STRING DEFAULT 'Canada';


SELECT
    s.* EXCEPT(over_18, pt, verdict) 
    , nt.rating_name
    , nt.primary_topic
    , nt.rating_short
    , slo.over_18
    , slo.allow_discovery
    , CASE 
        WHEN(COALESCE(slo.over_18, 'f') = 't') THEN 'over_18_or_X_M_D_V'
        WHEN(COALESCE(nt.rating_short, '') IN ('X', 'M', 'D', 'V')) THEN 'over_18_or_X_M_D_V'
        ELSE 'unrated_or_E'
    END AS grouped_rating

FROM `reddit-employee-datasets.david_bermejo.subclu_v0041_subreddit_clusters_c_a` AS t
    -- Inner join b/c we only want to keep subs that are geo-relevant AND in topic model
    INNER JOIN (
        SELECT *
        FROM `reddit-employee-datasets.david_bermejo.subclu_subreddit_geo_score_standardized_20220212`
        WHERE country_name = TARGET_COUNTRY
    ) AS s
        ON t.subreddit_id = s.subreddit_id

    -- Add rating so we can get an estimate for how many we can actually use for recommendation
    LEFT JOIN (
        SELECT *
        FROM `data-prod-165221.ds_v2_postgres_tables.subreddit_lookup`
        -- Get latest partition
        WHERE dt = DATE(CURRENT_DATE() - 2)
    ) AS slo
    ON s.subreddit_id = slo.subreddit_id
    LEFT JOIN (
        SELECT * FROM `data-prod-165221.cnc.shredded_crowdsource_topic_and_rating`
        WHERE pt = DATE(CURRENT_DATE() - 2)
    ) AS nt
        ON s.subreddit_id = nt.subreddit_id

    -- Exclude popular US subreddits
    -- Can't query this table from local notebook because of errors getting google drive permissions. smh, excludefor now
    LEFT JOIN `reddit-employee-datasets.david_bermejo.subclu_subreddits_top_us_to_exclude_from_relevance` tus
        ON s.subreddit_name = LOWER(tus.subreddit_name)

WHERE 1=1
    AND s.subreddit_name != 'profile'
    AND COALESCE(s.type, '') = 'public'
    AND COALESCE(s.verdict, 'f') <> 'admin_removed'
    AND COALESCE(slo.over_18, 'f') = 'f'
    AND COALESCE(nt.rating_short, '') NOT IN ('X', 'D')

    AND(
        s.geo_relevance_default = TRUE
        OR s.relevance_percent_by_subreddit = TRUE
        OR s.relevance_percent_by_country_standardized = TRUE
    )
    AND country_name IN (
            TARGET_COUNTRY
        )

    AND (
         -- Exclude subs that are top in US but we want to exclude as culturally relevant
         --  For simplicity, let's go with the English exclusion (more relaxed) than the non-English one
         COALESCE(tus.english_exclude_from_relevance, '') <> 'exclude'
    )

ORDER BY e_users_percent_by_country_standardized DESC, users_l7 DESC, subreddit_name
;

CPU times: user 798 ms, sys: 57.4 ms, total: 855 ms
Wall time: 16.1 s


### Check df with geo + language information

In [ ]:
print(df_geo_and_lang.shape)

(3204, 26)


In [ ]:
df_geo_and_lang.iloc[:4, :9]

,subreddit_id,subreddit_name,country_name,geo_relevance_default,b_users_percent_by_subreddit,e_users_percent_by_country_standardized,c_users_percent_by_country,d_users_percent_by_country_rank,relevance_percent_by_subreddit
0,t5_2raht,edmonton,Canada,True,0.716139,10.778536,0.012173,47,True
1,t5_2qi63,toronto,Canada,True,0.873624,10.764729,0.013378,37,True
2,t5_2qhov,vancouver,Canada,True,0.697738,10.762642,0.018740,20,True
3,t5_2qxon,alberta,Canada,True,0.654511,10.741714,0.011596,51,True


## Load model labels (clusters)

The clusters now live in a Big Query table and have standardized names, so pull the data from there.

### SQL labels


In [ ]:
%%time
%%bigquery df_labels --project data-science-prod-218515 

-- select subreddit clusters from bigQuery

SELECT
    sc.subreddit_id
    , sc.subreddit_name
    , nt.primary_topic

    , sc.* EXCEPT(subreddit_id, subreddit_name, primary_topic_1214)
FROM `reddit-employee-datasets.david_bermejo.subclu_v0041_subreddit_clusters_c_a` sc
    LEFT JOIN (
        -- New view should be visible to all, but still comes from cnc_taxonomy_cassandra_sync
        SELECT * FROM `data-prod-165221.cnc.shredded_crowdsource_topic_and_rating`
        WHERE DATE(pt) = (CURRENT_DATE() - 2)
    ) AS nt
        ON sc.subreddit_id = nt.subreddit_id
;

CPU times: user 8.92 s, sys: 562 ms, total: 9.48 s
Wall time: 1min 3s


### Check label outputs

In [ ]:
print(df_labels.shape)
df_labels.iloc[:4, :9]

(49558, 51)


,subreddit_id,subreddit_name,primary_topic,model_sort_order,posts_for_modeling_count,k_0013_label,k_0023_label,k_0041_label,k_0059_label
0,t5_5a9iie,progonlydj,None,40079,1000,12,19,34,49
1,t5_2x9c7,googleplaymusic,Music,40080,31,12,19,34,49
2,t5_3jzsk,ravedj,Music,40081,1000,12,19,34,49
3,t5_2rgie,happyhardcore,Music,40082,152,12,19,34,49


In [ ]:
counts_describe(df_labels.iloc[:, :9])

,dtype,count,unique,unique-percent,null-count,null-percent
subreddit_id,object,"49,558","49,558",100.00%,0,0.00%
subreddit_name,object,"49,558","49,558",100.00%,0,0.00%
primary_topic,object,"40,714",52,0.13%,"8,844",17.85%
model_sort_order,int64,"49,558","49,558",100.00%,0,0.00%
posts_for_modeling_count,int64,"49,558",999,2.02%,0,0.00%
k_0013_label,int64,"49,558",13,0.03%,0,0.00%
k_0023_label,int64,"49,558",23,0.05%,0,0.00%
k_0041_label,int64,"49,558",41,0.08%,0,0.00%
k_0059_label,int64,"49,558",59,0.12%,0,0.00%


# Reshape data
Apply reshaping fxns so that we can export the data in a format that's good for QA.

## Keep only labels for Target subreddits


In [ ]:
%%time
df_labels_target = keep_only_target_labels(
    df_labels=df_labels,
    df_geo=df_geo_and_lang,
    col_sort_order='model_sort_order',
    l_ix_subs=['subreddit_id', 'subreddit_name'],
    l_cols_to_front=None,
    geo_cols_to_drop=None,
)

0 <- subs to drop b/c they're not in model
(3204, 74) <- df_labels_target.shape
CPU times: user 229 ms, sys: 590 µs, total: 230 ms
Wall time: 229 ms


In [ ]:
counts_describe(df_labels_target.iloc[:, :15])

,dtype,count,unique,unique-percent,null-count,null-percent
model_sort_order,int64,"3,204","3,204",100.00%,0,0.00%
subreddit_id,object,"3,204","3,204",100.00%,0,0.00%
subreddit_name,object,"3,204","3,204",100.00%,0,0.00%
primary_topic,object,"2,815",51,1.81%,389,12.14%
rating_short,object,"3,139",3,0.10%,65,2.03%
over_18,object,804,1,0.12%,"2,400",74.91%
rating_name,object,"3,139",3,0.10%,65,2.03%
posts_for_modeling_count,int64,"3,204",730,22.78%,0,0.00%
k_0013_label,int64,"3,204",12,0.37%,0,0.00%
k_0023_label,int64,"3,204",21,0.66%,0,0.00%


## Run loop to find "optimal" min_num of subreddits for dynamic clusters


We want to balance two things:
- prevent orphan subreddits
- prevent clusters that are too large to be meaningful

In order to do this at a country level, we'll be better off starting with smallest cluster size and roll up until we have at least N subreddits in one cluster.

Find optimal `min_subreddits_in_cluster` based on:
- `orphan count`, 
- `number of clusters`,
- & other info

number might be different for each country and even within a country it might differ by when we filter NSFW subs.

### Loop

In [ ]:
%%time

col_new_cluster_val = 'cluster_label'
col_new_cluster_name = 'cluster_label_k'
col_new_cluster_prim_topic = 'cluster_majority_primary_topic'
col_new_cluster_topic_mix = 'cluster_topic_mix'

df_optimal_min_check = get_table_for_optimal_dynamic_cluster_params(
        df_labels_target=df_labels_target,
        col_new_cluster_val=col_new_cluster_val,
        col_new_cluster_name=col_new_cluster_name,
        col_new_cluster_prim_topic=col_new_cluster_prim_topic,
        col_new_cluster_topic_mix=col_new_cluster_topic_mix,
        min_subs_in_cluster_list=np.arange(3, 11),
        verbose=False,
)

100%|██████████| 16/16 [00:01<00:00, 13.27it/s]

100%|██████████| 16/16 [00:01<00:00, 13.60it/s]

100%|██████████| 16/16 [00:01<00:00, 13.26it/s]

100%|██████████| 16/16 [00:01<00:00, 13.52it/s]

100%|██████████| 16/16 [00:01<00:00, 13.65it/s]

100%|██████████| 16/16 [00:02<00:00,  6.88it/s]

100%|██████████| 16/16 [00:01<00:00, 13.88it/s]

100%|██████████| 16/16 [00:01<00:00, 13.72it/s]

100%|██████████| 8/8 [00:19<00:00,  2.45s/it]

CPU times: user 18.3 s, sys: 258 ms, total: 18.6 s
Wall time: 19.7 s


### Display loop results

In [ ]:
def highlight_below_threshold(val, threshold=1):
    if val <= threshold:
        return "color:purple; font-weight: bold; background-color:yellow;"
    else:
        return ''

col_num_orph_subs = 'num_orphan_subreddits'
# col_num_subs_mean = 'num_subreddits_per_cluster_mean'
col_num_subs_median = 'num_subreddits_per_cluster_median'

style_df_numeric(
    df_optimal_min_check,
    rename_cols_for_display=True,
    l_bar_simple=[col_num_orph_subs,
                  col_num_subs_median,]
).applymap(highlight_below_threshold, subset=[col_num_orph_subs.replace('_', ' ')])


,subs to cluster count,min subreddits in cluster,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,"3,204",3,548,2,5.85,5.00,24,"0001, 0012"
1,"3,204",4,460,1,6.97,6.00,19,0011
2,"3,204",5,388,1,8.26,7.00,15,0009
3,"3,204",6,334,1,9.59,9.00,16,0002
4,"3,204",7,297,0,10.79,10.00,12,
5,"3,204",8,265,0,12.09,11.00,10,
6,"3,204",9,239,0,13.41,12.00,9,
7,"3,204",10,219,0,14.63,13.00,9,


## Get dyanimc clusters (apply optimal num from above)

side bar: about 57% of subreddits in Australia only had a single primary topic as their `topic_mix`, so combining `primary topic` might not give us as much info as we hoped.

At the same time, for 43% of subs we might get additional detail by combining the primary topics.


In [ ]:
n_min_subs_in_cluster_optimal = 7

n_mix_start = 4
l_ix = ['subreddit_id', 'subreddit_name']
col_new_cluster_topic_mix = 'cluster_topic_mix'
col_subreddit_topic_mix = 'subreddit_full_topic_mix'
col_full_depth_mix_count = 'subreddit_full_topic_mix_count'
suffix_new_topic_mix = '_topic_mix_nested'

# even if cluster at k < 20 is generic, keep it to avoid orphan subs
l_cols_labels = (
    [c for c in df_labels_target.columns
        if all([c != col_new_cluster_val, c.endswith('_label')])
        ]
)

df_labels_target_dynamic_raw = create_dynamic_clusters(
    df_labels_target,
    agg_strategy='aggregate_small_clusters',
    min_subreddits_in_cluster=n_min_subs_in_cluster_optimal,
    l_cols_labels_input=l_cols_labels,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    n_mix_start=n_mix_start,
    col_new_cluster_topic_mix=col_new_cluster_topic_mix,
    col_subreddit_topic_mix=col_subreddit_topic_mix,
    col_full_depth_mix_count=col_full_depth_mix_count,
    suffix_new_topic_mix=suffix_new_topic_mix,
    l_ix=l_ix,
    verbose=True,
)

02:45:23 | INFO | "Concat'ing nested cluster labels..."
02:45:23 | INFO | "Getting topic mix at different depths..."
02:45:23 | INFO | "  Assigning base topic mix cols"
02:45:23 | INFO | "  Creating deepest base topic mix col..."
02:45:24 | INFO | "  Iterating through additional subs with multiple topics..."
100%|██████████| 16/16 [00:02<00:00,  6.02it/s]
02:45:27 | INFO | "Initializing values for strategy: aggregate_small_clusters"
02:45:27 | INFO | "  Looping to roll-up clusters from smallest to largest..."
100%|██████████| 21/21 [00:00<00:00, 71.38it/s]
02:45:27 | INFO | "(3204, 124) <- output shape"


In [ ]:
# [c for c in df_labels_target_dynamic_raw.columns if 'nested' in c]

In [ ]:
style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_raw,
        return_dict=False,
    ),
    rename_cols_for_display=True,
)

,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,297,0,10.79,10.00,12,


In [ ]:
# # check column order
style_df_numeric(
    df_labels_target_dynamic_raw.iloc[104:109, :15],
    rename_cols_for_display=True,
    int_labels=['total_users_in', 'num_of_countries_', 'users_in_subreddit_from_country_l28',
                    'by_country_rank',
                    ],
    pct_cols=['b_users_percent_by_subreddit',
                  'c_users_percent_by_country',
                  'users_percent_by_country_avg',
                  ],
    pct_labels='',
)

,subreddit id,subreddit name,cluster topic mix,primary topic,rating short,subreddit full topic mix,rating name,over 18,geo relevance default,relevance percent by subreddit,relevance percent by country standardized,b users percent by subreddit,e users percent by country standardized,d users percent by country rank,model sort order
104,t5_2rec7,autorepair,Cars and Motor Vehicles,Cars and Motor Vehicles,E,Cars and Motor Vehicles,Everyone,-,False,False,True,9.37%,2.37,"13,645","17,618"
105,t5_2vsnf,askmechanics,Cars and Motor Vehicles,Learning and Education,E,Cars and Motor Vehicles,Everyone,-,False,False,True,9.44%,2.82,"6,125","17,621"
106,t5_2rqxu,mechanics,Cars and Motor Vehicles,Careers,E,Cars and Motor Vehicles,Everyone,-,False,False,True,9.21%,2.52,"11,757","17,622"
107,t5_2rqqf,askamechanic,Cars and Motor Vehicles,Cars and Motor Vehicles,E,Cars and Motor Vehicles,Everyone,-,False,False,True,9.48%,2.53,"33,937","17,623"
108,t5_2tkvu,mechanicadvice,Cars and Motor Vehicles,Cars and Motor Vehicles,E,Cars and Motor Vehicles,Everyone,-,False,False,True,9.76%,3.30,503,"17,624"


### Minor QA checks

In [ ]:
# # check column order
# style_df_numeric(
#     df_labels_target_dynamic_raw.iloc[70:74, -22:],
#     rename_cols_for_display=True,
#     int_labels=['total_users_in', 'num_of_countries_', 'users_in_subreddit_from_country_l28',
#                     'by_country_rank',
#                     ],
#     pct_cols=['b_users_percent_by_subreddit',
#                   'c_users_percent_by_country',
#                   'users_percent_by_country_avg',
#                   ],
#     pct_labels='',
# )

In [ ]:
value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_new_cluster_topic_mix],
    top_n=9,
)

,cluster_topic_mix-count,cluster_topic_mix-percent,cluster_topic_mix-pct_cumulative_sum
"Business, Economics, and Finance",166,5.2%,5.2%
Cars and Motor Vehicles,150,4.7%,9.9%
Place,147,4.6%,14.5%
Food and Drink,146,4.6%,19.0%
Television | Podcasts and Streamers,144,4.5%,23.5%
Sports,126,3.9%,27.4%
Medical and Mental Health,126,3.9%,31.4%
Animals and Pets,114,3.6%,34.9%
Learning and Education,114,3.6%,38.5%


In [ ]:
# how many final clusters have multiple topics?
value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_new_cluster_topic_mix].str.count('\|')
)

,cluster_topic_mix-count,cluster_topic_mix-percent,cluster_topic_mix-pct_cumulative_sum
0,"1,981",61.8%,61.8%
1,919,28.7%,90.5%
2,270,8.4%,98.9%
3,34,1.1%,100.0%


In [ ]:
# how many SUBREDDITS have multiple topics? (when we check the deepest clusters)
#  these two calls are equivalent

# value_counts_and_pcts(
#     df_labels_target_dynamic_raw[col_subreddit_topic_mix].str.count('\|')
# )

value_counts_and_pcts(
    df_labels_target_dynamic_raw[col_full_depth_mix_count]
)

,subreddit_full_topic_mix_count-count,subreddit_full_topic_mix_count-percent,subreddit_full_topic_mix_count-pct_cumulative_sum
1,"1,604",50.1%,50.1%
2,937,29.2%,79.3%
3,490,15.3%,94.6%
4,147,4.6%,99.2%
5,26,0.8%,100.0%


In [ ]:
style_df_numeric(
    df_labels_target_dynamic_raw
    [df_labels_target_dynamic_raw[col_full_depth_mix_count] >= 5]
    .iloc[-5:, :9]
    ,
    rename_cols_for_display=True,
)

,subreddit id,subreddit name,cluster topic mix,primary topic,rating short,subreddit full topic mix,rating name,over 18,geo relevance default
1502,t5_2xinb,outoftheloop,Mature Themes and Adult Content | Gender,Learning and Education,E,Mature Themes and Adult Content | Gender | Funny/Humor | Celebrity | Learning and Education,Everyone,-,False
2470,t5_30ahy,quadrigacx,Science,Crypto,M,"Science | Mature Themes and Adult Content | Music | Business, Economics, and Finance | Crypto",Mature,-,True
3171,t5_2qmnw,halloween,Art,-,E,Art | Gaming | Television | Movies | Internet Culture and Memes,Everyone,f,False
3172,t5_2v9ac,beastwars,Art,Hobbies,E,Art | Gaming | Television | Internet Culture and Memes | Hobbies,Everyone,-,False
3203,t5_2y782o,get_coursehero_chegg,Music | Mature Themes and Adult Content | Internet Culture and Memes,Learning and Education,E,Music | Mature Themes and Adult Content | Internet Culture and Memes | Meta/Reddit | Learning and Education,Everyone,-,False


## Re-assign orphan subreddits (optional)

If there are subreddits that are orphan (see summary above), check them out to see if we can re-assign them w/o too much work. if we can't skip and move to the next country.

In [ ]:
# # check subs around orphan sub
# # n_plus_minus_ = 5
# ix_orphan_ = (
#     df_labels_target_dynamic_raw
#     [df_labels_target_dynamic_raw[col_new_cluster_val] == '0011']
#     .index
# )
# df_labels_target_dynamic_raw.iloc[ix_orphan_, :9]

In [ ]:
# # check other subs that are in the same cluster as orphan sub (at broadest level)
# l_cols_orphan_check = (
#     [
#         'subreddit_id',
#         col_new_cluster_topic_mix, 
#         # col_new_cluster_val,  # this can be really long and makes comparing harder
#         # col_subreddit_topic_mix,
#         'subreddit_name', 
#         col_new_cluster_name
#     ] +
#     l_cols_labels[:-5]
# )

# style_df_numeric(
#     df_labels_target_dynamic_raw
#     [df_labels_target_dynamic_raw['k_0013_label'] == 11]
#     [l_cols_orphan_check]
#     .iloc[3:14, :50]
#     ,
#     l_bar_simple=[c for c in l_cols_orphan_check[4:] if c.endswith('_label')],
#     rename_cols_for_display=True,

# )

In [ ]:
# label_k_to_reassign_ = 'k_0320_label'
# label_val_to_reassign_ = '0011-0018-0032-0043-0046-0058-0062-0087-0244'
# subreddit_id_orphan_ = 't5_2tt7r'

# mask_orphan_and_new_group = (
#     (df_labels_target_dynamic_raw['subreddit_id'] == subreddit_id_orphan_) |
#     (
#         (df_labels_target_dynamic_raw[col_new_cluster_name] == label_k_to_reassign_) &
#         (df_labels_target_dynamic_raw[col_new_cluster_val] == label_val_to_reassign_)
#     )
# )

# # assign is similar to what we do in the dynamic function
# label_k_new_ = 'k_0118_label'
# label_val_new_col_ = f"{label_k_new_}_nested"
# new_prim_topic_col_ = label_k_new_.replace('_label', '_majority_primary_topic')
# c_update_topic_mix_ = label_k_new_.replace('_label', suffix_new_topic_mix)

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_name
# ] = label_k_new_

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_val
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][label_val_new_col_]

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_prim_topic
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][new_prim_topic_col_]

# df_labels_target_dynamic_raw.loc[
#     mask_orphan_and_new_group,
#     col_new_cluster_topic_mix
# ] = df_labels_target_dynamic_raw[mask_orphan_and_new_group][c_update_topic_mix_]

# del mask_orphan_and_new_group, label_k_to_reassign_, label_val_to_reassign_
# del label_k_new_, label_val_new_col_, new_prim_topic_col_

In [ ]:
# check again, num of orphans should be lower than before
style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_raw,
        return_dict=False,
    ),
    rename_cols_for_display=True,
)

,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,388,1,8.26,7.00,15,0009


In [ ]:
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    ['cluster_label'],
    top_n=None,
    return_df=True
)['count'].describe()

count    388.000000
mean       8.255155
std        4.862380
min        1.000000
25%        6.000000
50%        7.000000
75%        9.000000
max       63.000000
Name: count, dtype: float64

## Get cluster for humans (list of subs in a cluster in a cell)
Here we get 1 cluster per row. 
Use cases:
- It makes it easier to quickly check NSFW clusters that we'll filter out
- we'll append the list of subreddit names from here to the final table for QA (makes it easier to evaluate whether the cluster makes sense).


In [ ]:
col_subs_in_cluster_count = 'subs_in_cluster_count'
col_list_cluster_names = 'list_cluster_subreddit_names'

df_cluster_for_humans = reshape_df_to_get_1_cluster_per_row(
    df_labels_target_dynamic_raw,
    col_counterpart_count=col_subs_in_cluster_count,
    col_list_cluster_names=col_list_cluster_names,
    col_list_cluster_ids='list_cluster_subreddit_ids',
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_topic=col_new_cluster_topic_mix,
    verbose=False,
    get_one_column_per_sub_id=False,
)

(297, 6)


In [ ]:
df_cluster_for_humans.iloc[40:46, 1:]

,cluster_label_k,cluster_topic_mix,subs_in_cluster_count,list_cluster_subreddit_ids,list_cluster_subreddit_names
40,k_0118_label,Marketplace and Deals,19,"t5_32st9, t5_hxrct, t5_2sacv, t5_qf7af, t5_31b0j, t5_2fvwbs, t5_2vpf3, t5_2r64t, t5_2xcmc, t5_2y0d6, t5_w6d7w, t5_2hzadd, t5_34nxp, t5_2r3oj, t5_27dv2p, t5_1pnmt1, t5_10rjzg, t5_2wf14, t5_3dzgjm","canadianhardwareswap, watchexchangecanada, geartrade, canadianknifeswap, airsoftmarketcanada, acnhturnips, acturnips, neopets, actrade, buddycrossing, officialhutrep, entalert, augustsmartlock, locksmith, courseherocheap, pcoptimum, mis..."
41,k_0320_label,Gaming | Outdoors and Nature,11,"t5_3mscn, t5_34ixl, t5_2ske1, t5_3bz54, t5_2qkvl, t5_10x52y, t5_10xt5n, t5_10xyj1, t5_2qi0i, t5_2qt7t, t5_3k0ii","mcmansionhell, cozyplaces, roomporn, ontarioabandoned, urbanexploration, edmontonwxrecords, ottawawxrecords, stjohnswxrecords, weather, canadia, casualcanada"
42,k_0118_label,Gaming | Sports,12,"t5_2u9o9, t5_3ft56, t5_2vldp, t5_3j8ny, t5_3ieex, t5_3fw0a, t5_2qk80, t5_3gkuw, t5_2sjy6, t5_3095o, t5_2t797, t5_2rbz0","runninglifestyle, xxrunning, carcamping, hottenting, wintercamping, campingalberta, camping, idiotsinboats, lifeguards, canoecamping, canoeing, kayaking"
43,k_1065_label,Gaming | Sports | Fitness and Nutrition,8,"t5_2ba9fa, t5_2qyi0, t5_2udvm, t5_2yxpt, t5_2iqfxq, t5_39us7, t5_2gnjmd, t5_2tat5","nordictrack, treadmills, bowflex, ifit, go_echelon, pelotoncycle, freestylelibre, fitbit"
44,k_0657_label,Gaming | Sports,11,"t5_2qhcs, t5_3ldvr, t5_2vk3g, t5_2n3v5u, t5_2t7jz, t5_2tf80, t5_2vs1b, t5_2sqb7, t5_2r1ig, t5_2sm0o, t5_2qx3n","golf, everybodysgolf, mariogolf, pga2k21, hockeygear, hockeygoalies, hockeyplayers, hockeyrefs, lacrosse, cribbage, foosball"
45,k_2207_label,Gaming | Sports,8,"t5_3dx9hs, t5_2zdea, t5_2skxr, t5_2tb1e, t5_2qi0t, t5_2tkx2, t5_3gip3, t5_2rguz","powsurf, snowskate, snowskating, spliddit, snowboarding, shredditgirls, snowboardingnoobs, iceskating"


In [ ]:
# df_cluster_for_humans.tail(9)

### Check subs that have mature topics

Make a list of sub IDs to exclude for clean df

In [ ]:
mask_mature_clusters_ = (
    df_cluster_for_humans[col_new_cluster_topic_mix].str.lower()
    .str.contains('mature')
)
print(f"{mask_mature_clusters_.sum()} <- clusters with 'Mature' in topic mix")

20 <- clusters with 'Mature' in topic mix


In [ ]:
(
    df_cluster_for_humans
    [mask_mature_clusters_]
    .iloc[:11, :]
)

,cluster_label,cluster_label_k,cluster_topic_mix,subs_in_cluster_count,list_cluster_subreddit_ids,list_cluster_subreddit_names
0,0001-0001,k_0023_label,Mature Themes and Adult Content,17,"t5_38twja, t5_46yyke, t5_4zfh9h, t5_he7gg, t5_29nix8, t5_2xbogx, t5_25dzon, t5_2rd2l, t5_3prv4, t5_13wr2b, t5_54c1dr, t5_4gyc88, t5_42c6wo, t5_3lbae, t5_twbnh, t5_2zui0, t5_59yqk7","punjabiactresses, faptopunjabiactresses, punjabitopmodels, jackieredmond, canadianbabes, maddisonnoel, irlbeauty, drama, cestvrai_, alexannlp, joyli, brooklyntrapp, saharrooo, claudiatihan, beatricebouchard, eugeniebouchard, billiebopbi..."
1,0001-0001-0003-0003-0003-0004-0005-0005,k_0118_label,Mature Themes and Adult Content,10,"t5_52u94h, t5_25ejbo, t5_3a8nhj, t5_2gayb5, t5_4tyhj0, t5_56iabp, t5_2nx6ct, t5_53ddi5, t5_37mdgs, t5_36bvt","mikaylademaiternew, hailiedeegan19, sjbleau, camillefelton, liviamartinnsfw, okanaganbabes, lexiiwiinters, nimratkhaira, barstoolboners2, skyesweetnam"
2,0002,k_0013_label,Mature Themes and Adult Content,8,"t5_4169ot, t5_54r07k, t5_4x4glj, t5_4o7kt0, t5_2w5lk, t5_3m98i4, t5_57mhli, t5_38rxz","juliamerekouliasmedia, alyssakulaninsfw, sylviaganijerk, jaydecandanceofv2, dannieriel, melika_dvr, gennabartonleaks, amulet"
3,0004-0006-0011-0013-0013,k_0063_label,Mature Themes and Adult Content,9,"t5_52vpfg, t5_512v3f, t5_59o53r, t5_4wwf1s, t5_rl58z, t5_2tdoo, t5_2s0wq, t5_2sypn, t5_38ezr","barriesdirtysecrets, winnipeghookup, bramptonhookups, torontohangoutfriends, vancouver4friends, calgarysocialclub, torontoevents, r4rcanada, amwfdating"
4,0005,k_0013_label,Mature Themes and Adult Content,4,"t5_333yu, t5_2vcez, t5_2tsp9, t5_fdto3","trollingforababy, nonbinary, thechive, victoriabaldesarra"
132,0008-0013-0023-0032-0034-0044-0046-0063-0177-0349,k_0657_label,Mature Themes and Adult Content | Family and Relationships,12,"t5_2qipv, t5_34cyw, t5_2qpe9, t5_2vfzu, t5_30m5u, t5_2c0k6s, t5_26qn95, t5_39x1d, t5_2fm42g, t5_xj4n6, t5_2t25p, t5_2qu5n","divorce, datingoverthirty, onlinedating, thegirlsurvivalguide, askwomenadvice, femalelevelupstrategy, ualbertacrush, muslimmarriage, muslimmarriage2, muslimfamilysolutions, deadbedrooms, polyamory"
133,0008-0013-0023-0032-0034-0044-0046-0064,k_0118_label,Mature Themes and Adult Content | Gender,18,"t5_mwfyw, t5_3n04k0, t5_2tg6f, t5_2rfyw, t5_30c2m, t5_38jid, t5_2sk0u, t5_xaiot, t5_2w271, t5_2qkeh, t5_2sp85, t5_2rxrw, t5_2tm6f, t5_3531l, t5_35hao, t5_2xinb, t5_296zi1, t5_39xf0","latebloomerlesbians, montreallgbt, asiantwox, asianamerican, asianmasculinity, menslib, malelifestyle, femaledatingstrategy, askacanadian, answers, explainlikeimcalvin, askwomen, askredddit, bumble, hingeapp, outoftheloop, smolbeansnark..."
134,0008-0013-0023-0032-0034-0044-0046-0064-0181,k_0320_label,Mature Themes and Adult Content | Gender,8,"t5_2t187, t5_2r4b9, t5_31hoq, t5_2t9mw, t5_h1sjl, t5_32zxv, t5_3dd6l, t5_2r1c3","mypartneristrans, asktransgender, ask_transgender, ftm, ftmmen, transvancouver, transontario, genderqueer"
135,0008-0013-0023-0032-0034-0044-0046-0064-0184-0362-0525-0588,k_1065_label,Mature Themes and Adult Content | Gender | Family and Relationships,9,"t5_nhaha, t5_21xpjt, t5_2vplf, t5_2qhtr, t5_2rv3t, t5_2ug26, t5_11080t, t5_2w13j, t5_2ra25","weddingshaming, weddingscanada, weddingsunder10k, wedding, weddingplanning, engaged, waiting_to_wed, justengaged, engagementrings"
153,0008-0014-0025-0034-0036-0046-0048-0066,k_0118_label,Mature Themes and Adult Content,10,"t5_2rn3k, t5_3j815, t5_2vnoe, t5_2wa9rp, t5_56htpm, t5_2uooe, t5_2qszq, t5_325x2, t5_2yt3g, t5_3jpgqe","adderall, concerta, biohackers, canadiansippers, psycspotlight, psilocybin, shrooms, psilocybinmushrooms, microdosing, psychedelicmomscanada"


In [ ]:
(
    df_cluster_for_humans
    [mask_mature_clusters_]
    .iloc[-10:, :]
)

,cluster_label,cluster_label_k,cluster_topic_mix,subs_in_cluster_count,list_cluster_subreddit_ids,list_cluster_subreddit_names
154,0008-0014-0025-0034-0036-0046-0048-0066-0189-0374-0541-0608-0884-1038-1250-1327-1589-1774-1903-2070-2159-2190,k_3927_label,Mature Themes and Adult Content | Medical and Mental Health,8,"t5_xuccg, t5_2dmvki, t5_33h8h, t5_u438e, t5_37a9h, t5_2x77y, t5_31mz6, t5_2eqx39","bupropion, wellbutrin_bupropion, effexor, effexorsuccess, depressionregimens, zoloft, lexapro, trintellix"
155,0008-0014-0025-0034-0036-0047-0049-0067,k_0118_label,Mature Themes and Adult Content,11,"t5_2qhg1, t5_32t1p, t5_2tqqt, t5_gs5wx, t5_38arj, t5_2rar5, t5_2t7is, t5_qjekx, t5_2tclf, t5_2rukl, t5_2r9vp","beer, torontocraftbeer, beercanada, vancouvercraftbeer, biereqc, rootbeer, canadients, edmontonmarijuana, eldertrees, saplings, trees"
156,0008-0014-0025-0034-0036-0047-0049-0067-0191-0380,k_0657_label,Mature Themes and Adult Content | Food and Drink,8,"t5_2qq65, t5_3p41mb, t5_34k59, t5_2qpco, t5_2ri8c, t5_2qhs8, t5_2rmhu, t5_2qi6q","scotch, bcwhiskey, canadawhisky, cocktails, tequila, wine, absinthe, alcohol"
157,0008-0014-0025-0034-0036-0047-0049-0067-0192-0381-0551-0618-0898-1054-1270,k_2207_label,Mature Themes and Adult Content,11,"t5_58491c, t5_pyh1z, t5_2tdxd, t5_q003x, t5_3ij9g, t5_u8o2x, t5_q0suy, t5_2tv6j, t5_3i30o, t5_2tlbm, t5_eydl8","ocsreviews4theculture, recpics, entottawa, sqdc, canadiancannabislps, thebccs, theocs, vancouverents, canadianmoms, montrealents, jahseeds"
158,0008-0014-0025-0034-0036-0047-0049-0067-0193-0382-0552-0620-0901,k_1560_label,Mature Themes and Adult Content,8,"t5_2rhtf, t5_2slm7, t5_37k43, t5_2th0s, t5_3axkx, t5_2ydw1, t5_2r341, t5_vko1y","bongs, stonerengineering, chinaglass, dabs, dailydabbers, bubblehash, cannabisextracts, cracksmokers"
159,0008-0014-0025-0034-0036-0047-0049-0067-0193-0383-0553-0621-0902-1059,k_1840_label,Mature Themes and Adult Content,8,"t5_2trc3, t5_37acg, t5_lc6ci, t5_3cny9, t5_2tkz8, t5_2u7ap, t5_3b8j4, t5_13v3gl","davidstea, backwoods, backwoodsmarket, cigarscanada, haligonients, incense, juul, stlth"
160,0008-0014-0025-0034-0036-0047-0049-0067-0193-0383-0554-0622-0903,k_1560_label,Mature Themes and Adult Content,8,"t5_2go4wi, t5_2yvrn, t5_2tuh9, t5_2tljh, t5_3cxz8, t5_30gcq, t5_36bgk, t5_2upjd","vapingcanada, canadian_ecigarette, arizer, volcanovaporizer, divinetribevaporizers, waxpen, paxvapor, vaporents"
204,0010-0017-0030-0040-0042-0053-0056-0078,k_0118_label,Place | Mature Themes and Adult Content,11,"t5_2qmsc, t5_2wvvc, t5_2s5e8, t5_38ve9, t5_2qxgd, t5_31d2x, t5_3lozk, t5_2qqdb, t5_2vxxc, t5_2qjo5, t5_m564q","unsolvedmysteries, unresolvedmysteries, truecrime, truecrimediscussion, missingpersons, missing411, badcopnotimbit, bad_cop_no_donut, justiceserved, chaoticgood, imatotalpeiceofshit"
205,0010-0017-0030-0040-0042-0053-0056-0078-0219-0437-0630-0705,k_1065_label,Place | Mature Themes and Adult Content | Law,9,"t5_navml, t5_33vph, t5_2s3pg, t5_2eiusg, t5_51bf5n, t5_2cfkv9, t5_3mfaz, t5_fl4tn, t5_2ks21s","chriswatts, serialpodcast, jonbenetramsey, lorivallow, gabbypetito, epsteinandfriends, countingon, duggarssnark, jeremydewitte"
296,0013-0023-0041-0059-0063-0079-0085-0118-0316,k_0320_label,Music | Mature Themes and Adult Content | Internet Culture and Memes,9,"t5_oizrm, t5_2xmrc, t5_2schz, t5_112ivy, t5_3g55k, t5_3mjy6, t5_247h, t5_2qqnx, t5_2y782o","namenerdcirclejerk, namenerds, crossword, nytspellingbee, 99rideau, mompics, ca, albert, get_coursehero_chegg"


- model did a good job identifying this drug cluster... but yeah def don't want to increase their reach   
    - `0008-0014-0025-0034-0036-0046-0048-0066`
    - `0008-0014-0025-0034-0036-0046-0048-0066-0190-0378`

- the drinking might be SFW... but prob not the 420
    - `0008-0014-0025-0034-0036-0047-0049-0067`
    - `0008-0014-0025-0034-0036-0047-0049-0067-0193`

In [ ]:
# might need to re-import in order for updates to show up
from subclu.models.reshape_clusters_v041 import _L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_

_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_

['0001',
 '0001-0001',
 '0001-0001-0001',
 '0001-0001-0001-0001-0001-0001-0001-0001',
 '0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001-0001',
 '0001-0001-0001-0001-0001-0001-0001-0001-0001-0002-0002-0002-0002-0002-0002-0002-0003-0003-0003-0003-0003-0003',
 '0001-0001-0001-0001-0001-0001-0001-0001-0001-0002-0002-0002-0002-0002-0002-0002-0003-0003-0004',
 '0001-0001-0001-0001-0001-0001-0001-0001-0001-0002-0002-0002-0002-0002-0003-0003',
 '0001-0001-0001-0001-0001-0001-0001-0001-0001-0002-0002-0003-0003-0003-0004-0004-0006-0007-0009-0010-0010-0010',
 '0001-0001-0001-0001-0001-0001-0001-0001-0003-0005-0006-0007-0009-0009-0012-0012-0018-0020-0023-0026-0026-0026',
 '0001-0001-0001-0001-0001-0002-0002-0002',
 '0001-0001-0002-0002-0002-0003-0003-0003',
 '0001-0001-0003-0003-0003-0004-0005-0005',
 '0002',
 '0002-0002',
 '0002-0002-0005-0005-0005',
 '0002-0002-0005-0005-0005-0007-0008-0009',
 '0004',
 '0005',
 '0005-0007-0012',
 '0008-001

### Add the flag to exclude from QA & the list of sub names to df-raw

In [ ]:
val_exclude_from_qa_ = 'exclude from QA'
col_exclude_from_qa = 'exclude_from_qa'

df_cluster_for_humans[col_exclude_from_qa] = np.where(
    df_cluster_for_humans[col_new_cluster_val].isin(_L_MATURE_CLUSTERS_TO_EXCLUDE_FROM_QA_),
    val_exclude_from_qa_,
    'keep',  
)
print(f"{mask_mature_clusters_.sum()} <- clusters with 'Mature' in topic mix")
display(value_counts_and_pcts(df_cluster_for_humans[col_exclude_from_qa]))


df_labels_target_dynamic_raw = (
    df_labels_target_dynamic_raw
    # drop in case we're redoing the values
    .drop(
        [c for c in [col_list_cluster_names, col_subs_in_cluster_count, col_exclude_from_qa] if c in df_labels_target_dynamic_raw.columns],
        axis=1
    )
    .merge(
        df_cluster_for_humans[[col_new_cluster_val, col_list_cluster_names, col_subs_in_cluster_count, col_exclude_from_qa]],
        how='left',
        on=[col_new_cluster_val],
    )
)

df_labels_target_dynamic_raw.shape

20 <- clusters with 'Mature' in topic mix


,exclude_from_qa-count,exclude_from_qa-percent,exclude_from_qa-pct_cumulative_sum
keep,290,97.6%,97.6%
exclude from QA,7,2.4%,100.0%


(3204, 127)

In [ ]:
# df_labels_target_dynamic_raw.iloc[:5, -7:]

In [ ]:
mask_keep_clusters = (
    df_cluster_for_humans[col_exclude_from_qa] != val_exclude_from_qa_
)
(
    df_cluster_for_humans
    [mask_mature_clusters_ & mask_keep_clusters]
    .iloc[:7, :]
)

,cluster_label,cluster_label_k,cluster_topic_mix,subs_in_cluster_count,list_cluster_subreddit_ids,list_cluster_subreddit_names,exclude_from_qa
3,0004-0006-0011-0013-0013,k_0063_label,Mature Themes and Adult Content,9,"t5_52vpfg, t5_512v3f, t5_59o53r, t5_4wwf1s, t5_rl58z, t5_2tdoo, t5_2s0wq, t5_2sypn, t5_38ezr","barriesdirtysecrets, winnipeghookup, bramptonhookups, torontohangoutfriends, vancouver4friends, calgarysocialclub, torontoevents, r4rcanada, amwfdating",keep
132,0008-0013-0023-0032-0034-0044-0046-0063-0177-0349,k_0657_label,Mature Themes and Adult Content | Family and Relationships,12,"t5_2qipv, t5_34cyw, t5_2qpe9, t5_2vfzu, t5_30m5u, t5_2c0k6s, t5_26qn95, t5_39x1d, t5_2fm42g, t5_xj4n6, t5_2t25p, t5_2qu5n","divorce, datingoverthirty, onlinedating, thegirlsurvivalguide, askwomenadvice, femalelevelupstrategy, ualbertacrush, muslimmarriage, muslimmarriage2, muslimfamilysolutions, deadbedrooms, polyamory",keep
133,0008-0013-0023-0032-0034-0044-0046-0064,k_0118_label,Mature Themes and Adult Content | Gender,18,"t5_mwfyw, t5_3n04k0, t5_2tg6f, t5_2rfyw, t5_30c2m, t5_38jid, t5_2sk0u, t5_xaiot, t5_2w271, t5_2qkeh, t5_2sp85, t5_2rxrw, t5_2tm6f, t5_3531l, t5_35hao, t5_2xinb, t5_296zi1, t5_39xf0","latebloomerlesbians, montreallgbt, asiantwox, asianamerican, asianmasculinity, menslib, malelifestyle, femaledatingstrategy, askacanadian, answers, explainlikeimcalvin, askwomen, askredddit, bumble, hingeapp, outoftheloop, smolbeansnark...",keep
134,0008-0013-0023-0032-0034-0044-0046-0064-0181,k_0320_label,Mature Themes and Adult Content | Gender,8,"t5_2t187, t5_2r4b9, t5_31hoq, t5_2t9mw, t5_h1sjl, t5_32zxv, t5_3dd6l, t5_2r1c3","mypartneristrans, asktransgender, ask_transgender, ftm, ftmmen, transvancouver, transontario, genderqueer",keep
135,0008-0013-0023-0032-0034-0044-0046-0064-0184-0362-0525-0588,k_1065_label,Mature Themes and Adult Content | Gender | Family and Relationships,9,"t5_nhaha, t5_21xpjt, t5_2vplf, t5_2qhtr, t5_2rv3t, t5_2ug26, t5_11080t, t5_2w13j, t5_2ra25","weddingshaming, weddingscanada, weddingsunder10k, wedding, weddingplanning, engaged, waiting_to_wed, justengaged, engagementrings",keep
154,0008-0014-0025-0034-0036-0046-0048-0066-0189-0374-0541-0608-0884-1038-1250-1327-1589-1774-1903-2070-2159-2190,k_3927_label,Mature Themes and Adult Content | Medical and Mental Health,8,"t5_xuccg, t5_2dmvki, t5_33h8h, t5_u438e, t5_37a9h, t5_2x77y, t5_31mz6, t5_2eqx39","bupropion, wellbutrin_bupropion, effexor, effexorsuccess, depressionregimens, zoloft, lexapro, trintellix",keep
156,0008-0014-0025-0034-0036-0047-0049-0067-0191-0380,k_0657_label,Mature Themes and Adult Content | Food and Drink,8,"t5_2qq65, t5_3p41mb, t5_34k59, t5_2qpco, t5_2ri8c, t5_2qhs8, t5_2rmhu, t5_2qi6q","scotch, bcwhiskey, canadawhisky, cocktails, tequila, wine, absinthe, alcohol",keep


## Create new df_clean 

- Add list of subreddits to target-CLEAN, b/c we'll need it for rating final
- Add new columns & update order


### Copy baseline cols for clean

In [ ]:
col_model_sort_order = 'model_sort_order'

# create new col as int for label so we can add a color scale when doing QA
col_new_cluster_val_int = 'cluster_label_int'
df_labels_target_dynamic_raw[col_new_cluster_val_int] = (
    df_labels_target_dynamic_raw[col_new_cluster_val]
    .str[-4:].astype(int)
)

# create new column for reddit link, makes QA easier
col_link_to_sub = 'link_to_sub'
link_prefix_ = 'www.reddit.com/r/'
df_labels_target_dynamic_raw[col_link_to_sub] = link_prefix_ + df_labels_target_dynamic_raw['subreddit_name']

# create lookup for rating col, might need to refresh it later in google sheets, though
col_rated_e = 'rated E'
# need to make it boolean for checkbox thing to work
df_labels_target_dynamic_raw[col_rated_e] = (
    df_labels_target_dynamic_raw['rating_short'] == 'E'
)

# Set expected col order
l_cols_clean_final_for_qa = [
    'subreddit_id',
    'subreddit_name',
    col_new_cluster_val_int,
    col_new_cluster_topic_mix,
    
    # insert inputs for QA cols
    'not country relevant',
    col_rated_e,
    'relevant to cluster/ other subreddits in cluster',
    'safe to show in relation to cluster',
    col_link_to_sub,

    # cols for notes
    'country relevance notes',
    'rating or cluster notest',
    #  # 'cluster relevance notes',

    col_subs_in_cluster_count,
    col_list_cluster_names,

    'allow_discovery',
    'users_l7',
    'total_users_in_subreddit_l28',
    'rating_name',

    'over_18',
    'rating_short',
    'primary_topic',
    col_subreddit_topic_mix,
    # why did a sub get marked as geo or culturally relevant?
    #  can use them to sort
    'geo_relevance_default',
    'relevance_percent_by_subreddit',
    'relevance_percent_by_country_standardized',
    'b_users_percent_by_subreddit',
    'e_users_percent_by_country_standardized',
    'd_users_percent_by_country_rank',
    col_model_sort_order,

    'posts_for_modeling_count',
    'c_users_percent_by_country',
    'users_in_subreddit_from_country_l28',
    'total_users_in_country_l28',
    
    col_new_cluster_val,
    col_new_cluster_name,
    col_new_cluster_prim_topic,
]

# copy existing columns from raw + 
l_cols_clean_existing = [c for c in l_cols_clean_final_for_qa if c in df_labels_target_dynamic_raw.columns]
l_cols_clean_new = [c for c in l_cols_clean_final_for_qa if c not in df_labels_target_dynamic_raw.columns]

# reorder cols in raw DF so it's easier to switch between the two tabs
df_labels_target_dynamic_raw = df_labels_target_dynamic_raw[
    reorder_array(l_cols_clean_existing, df_labels_target_dynamic_raw.columns)
]

# only copy subs that aren't in excluded clusters!!
df_labels_target_dynamic_clean = (
    df_labels_target_dynamic_raw
    [df_labels_target_dynamic_raw[col_exclude_from_qa] != val_exclude_from_qa_]
    [l_cols_clean_existing]
    .copy()
)
# Add new columns and initialize them with empty strings
df_labels_target_dynamic_clean[l_cols_clean_new] = ''

# check num of orphans for clean
display(style_df_numeric(
    get_dynamic_cluster_summary(
        df_labels_target_dynamic_clean,
        return_dict=False,
    ),
    rename_cols_for_display=True,
).set_caption(f"Summary for df-CLEAN"))

# re-order & rename the columns so their easier to see in google sheets
# Sorty by cluster label b/c sometimes a sub won't be clustered dynamically next to closest neighbors!
df_labels_target_dynamic_clean = (
    df_labels_target_dynamic_clean[l_cols_clean_final_for_qa]
    .sort_values(by=[col_new_cluster_val, col_model_sort_order], ascending=True)
    # do final renaming only when saving, otherwise, it's a pain to adjust or look things up?
    .rename(columns={c: c.replace('_', ' ') for c in l_cols_clean_final_for_qa[5:]})
)

print(f"{len(l_cols_clean_final_for_qa)} <- expected final col count")
print(f"{df_labels_target_dynamic_raw.shape} <- df raw shape")
print(f"{df_labels_target_dynamic_clean.shape} <- df clean shape")

,cluster count,num orphan subreddits,num subreddits per cluster mean,num subreddits per cluster median,num clusters with mature primary topic,cluster ids with orphans
0,290,0,10.81,10.00,6,


35 <- expected final col count
(3204, 130) <- df raw shape
(3135, 35) <- df clean shape


In [ ]:
df_labels_target_dynamic_clean.iloc[:5, :24]

,subreddit_id,subreddit_name,cluster_label_int,cluster_topic_mix,not country relevant,rated E,relevant to cluster/ other subreddits in cluster,safe to show in relation to cluster,link to sub,country relevance notes,rating or cluster notest,subs in cluster count,list cluster subreddit names,allow discovery,users l7,total users in subreddit l28,rating name,over 18,rating short,primary topic,subreddit full topic mix,geo relevance default,relevance percent by subreddit,relevance percent by country standardized
35,t5_52vpfg,barriesdirtysecrets,13,Mature Themes and Adult Content,,False,,,www.reddit.com/r/barriesdirtysecrets,,,9,"barriesdirtysecrets, winnipeghookup, bramptonhookups, torontohangoutfriends, vancouver4friends, calgarysocialclub, torontoevents, r4rcanada, amwfdating",None,259,881,None,None,None,None,Mature Themes and Adult Content,True,True,True
36,t5_512v3f,winnipeghookup,13,Mature Themes and Adult Content,,False,,,www.reddit.com/r/winnipeghookup,,,9,"barriesdirtysecrets, winnipeghookup, bramptonhookups, torontohangoutfriends, vancouver4friends, calgarysocialclub, torontoevents, r4rcanada, amwfdating",None,327,1195,None,None,None,None,Mature Themes and Adult Content,True,True,True
37,t5_59o53r,bramptonhookups,13,Mature Themes and Adult Content,,False,,,www.reddit.com/r/bramptonhookups,,,9,"barriesdirtysecrets, winnipeghookup, bramptonhookups, torontohangoutfriends, vancouver4friends, calgarysocialclub, torontoevents, r4rcanada, amwfdating",None,149,524,None,None,None,None,Mature Themes and Adult Content,True,True,False
38,t5_4wwf1s,torontohangoutfriends,13,Mature Themes and Adult Content,,True,,,www.reddit.com/r/torontohangoutfriends,,,9,"barriesdirtysecrets, winnipeghookup, bramptonhookups, torontohangoutfriends, vancouver4friends, calgarysocialclub, torontoevents, r4rcanada, amwfdating",None,690,1980,Everyone,None,E,None,Mature Themes and Adult Content | Family and Relationships | Place,True,True,True
39,t5_rl58z,vancouver4friends,13,Mature Themes and Adult Content,,True,,,www.reddit.com/r/vancouver4friends,,,9,"barriesdirtysecrets, winnipeghookup, bramptonhookups, torontohangoutfriends, vancouver4friends, calgarysocialclub, torontoevents, r4rcanada, amwfdating",t,3658,8789,Everyone,f,E,Family and Relationships,Mature Themes and Adult Content | Family and Relationships | Place,True,True,True


## Create target to target list for FPR example

In [ ]:
%%time

df_target_to_target_list = convert_distance_or_ab_to_list_for_fpr(
    df_labels_target_dynamic_clean.rename(columns={c: c.replace(' ', '_') for c in df_labels_target_dynamic_clean.columns}),
    convert_to_ab=True,
    col_counterpart_count='counterpart_count',
    col_list_cluster_names='list_cluster_subreddit_names',
    col_list_cluster_ids='list_cluster_subreddit_ids',
    l_cols_for_seeds=None,
    l_cols_for_clusters=None,
    col_new_cluster_val=col_new_cluster_val,
    col_new_cluster_name=col_new_cluster_name,
    col_new_cluster_prim_topic=col_new_cluster_prim_topic,
    verbose=False,
)
df_target_to_target_list.shape

  (39352, 9) <- df_ab.shape after removing matches to self
  (3135, 7) <- df_a_to_b.shape
CPU times: user 285 ms, sys: 2.8 ms, total: 288 ms
Wall time: 383 ms


In [ ]:
df_target_to_target_list.iloc[:5, :11]

,subreddit_name_seed,subreddit_id_seed,cluster_label,cluster_label_k,counterpart_count,list_cluster_subreddit_names,list_cluster_subreddit_ids
0,barriesdirtysecrets,t5_52vpfg,0004-0006-0011-0013-0013,k_0063_label,8,"winnipeghookup, bramptonhookups, torontohangoutfriends, vancouver4friends, calgarysocialclub, torontoevents, r4rcanada, amwfdating","t5_512v3f, t5_59o53r, t5_4wwf1s, t5_rl58z, t5_2tdoo, t5_2s0wq, t5_2sypn, t5_38ezr"
1,winnipeghookup,t5_512v3f,0004-0006-0011-0013-0013,k_0063_label,8,"barriesdirtysecrets, bramptonhookups, torontohangoutfriends, vancouver4friends, calgarysocialclub, torontoevents, r4rcanada, amwfdating","t5_52vpfg, t5_59o53r, t5_4wwf1s, t5_rl58z, t5_2tdoo, t5_2s0wq, t5_2sypn, t5_38ezr"
2,bramptonhookups,t5_59o53r,0004-0006-0011-0013-0013,k_0063_label,8,"barriesdirtysecrets, winnipeghookup, torontohangoutfriends, vancouver4friends, calgarysocialclub, torontoevents, r4rcanada, amwfdating","t5_52vpfg, t5_512v3f, t5_4wwf1s, t5_rl58z, t5_2tdoo, t5_2s0wq, t5_2sypn, t5_38ezr"
3,torontohangoutfriends,t5_4wwf1s,0004-0006-0011-0013-0013,k_0063_label,8,"barriesdirtysecrets, winnipeghookup, bramptonhookups, vancouver4friends, calgarysocialclub, torontoevents, r4rcanada, amwfdating","t5_52vpfg, t5_512v3f, t5_59o53r, t5_rl58z, t5_2tdoo, t5_2s0wq, t5_2sypn, t5_38ezr"
4,vancouver4friends,t5_rl58z,0004-0006-0011-0013-0013,k_0063_label,8,"barriesdirtysecrets, winnipeghookup, bramptonhookups, torontohangoutfriends, calgarysocialclub, torontoevents, r4rcanada, amwfdating","t5_52vpfg, t5_512v3f, t5_59o53r, t5_4wwf1s, t5_2tdoo, t5_2s0wq, t5_2sypn, t5_38ezr"


# Export data

Note that we have to use `fillna('')`

Otherwise, we'll get errors because the gspread library doesn't know how to handle `pd.NaN` or `np.Nan` (nulls).

## Define variables to create/access google sheet doc & worksheets

In [ ]:
gspread.__version__

'4.0.1'

In [ ]:
# # %%time
os.environ['GOOGLE_CLOUD_PROJECT'] = 'data-prod-165221'
country_name_sheet_ = 'Canada'
target_abbrev_ = 'ca'
GSHEET_KEY = '1bRRcL-BBh2ngOCWS_SS9RjWr-lArOH4awCBnb80kVeA'
GSHEET_NAME = f'i18n {country_name_sheet_} subreddits and clusters - model v0.4.1'


d_wsh_names = {
    'qa_ready': {
        'name': 'subs_need_to_be_rated',
    },
    'sub_raw': {
        'name': 'raw_data_per_subreddit',
    },
    'clusters_t2t_list_raw': {
        'name': f'raw_clusters_list_{target_abbrev_}_{target_abbrev_}',
    },
    'clusters_t2t_fpr_raw': {
        'name': f'raw_clusters_fpr_{target_abbrev_}_{target_abbrev_}',
    }
}

if GSHEET_KEY is not None:
    sh = gc.open_by_key(GSHEET_KEY)
    print(f"Opening google worksheet: {GSHEET_NAME} ...")
else:
    print(f"Creating google worksheet: {GSHEET_NAME} ...")
    sh = gc.create(GSHEET_NAME)

# create worksheets:
for _, d_ in d_wsh_names.items():
    sh_name = d_['name']
    try:
        d_['worksheet'] = sh.worksheet(sh_name)
        print(f"Opening tab/sheet: {sh_name} ...")
    except Exception as e:
        print(f"Creating tab/sheet: {sh_name} ...")
        d_['worksheet'] = sh.add_worksheet(sh_name, rows=5, cols=5)


Opening google worksheet: i18n Canada subreddits and clusters - model v0.4.1 ...
Opening tab/sheet: subs_need_to_be_rated ...
Opening tab/sheet: raw_data_per_subreddit ...
Opening tab/sheet: raw_clusters_list_ca_ca ...
Opening tab/sheet: raw_clusters_fpr_ca_ca ...


## Save: Clean sheet to rate

In [ ]:
d_wsh_names.keys()

dict_keys(['qa_ready', 'sub_raw', 'clusters_t2t_list_raw', 'clusters_t2t_fpr_raw'])

In [ ]:
%%time
(
    d_wsh_names['qa_ready']['worksheet']
    .update([df_labels_target_dynamic_clean.columns.values.tolist()] + 
             df_labels_target_dynamic_clean.fillna('').values.tolist())
)

CPU times: user 81 ms, sys: 3.91 ms, total: 84.9 ms
Wall time: 3.58 s


{'spreadsheetId': '1bRRcL-BBh2ngOCWS_SS9RjWr-lArOH4awCBnb80kVeA',
 'updatedCells': 109760,
 'updatedColumns': 35,
 'updatedRange': 'subs_need_to_be_rated!A1:AI3136',
 'updatedRows': 3136}

## Save: df cluster for humans

In [ ]:
%%time

(
    d_wsh_names['clusters_t2t_list_raw']['worksheet']
    .update(
        [df_cluster_for_humans.columns.values.tolist()] + 
        df_cluster_for_humans.fillna('').values.tolist()
    )
)

CPU times: user 7.5 ms, sys: 0 ns, total: 7.5 ms
Wall time: 369 ms


{'spreadsheetId': '1bRRcL-BBh2ngOCWS_SS9RjWr-lArOH4awCBnb80kVeA',
 'updatedCells': 2086,
 'updatedColumns': 7,
 'updatedRange': 'raw_clusters_list_ca_ca!A1:G298',
 'updatedRows': 298}

## Save: target raw dynamic


In [ ]:
l_cols_to_drop = (
    ['table_creation_date'] +
    [c for c in df_labels_target_dynamic_raw.columns if c.endswith('_nested')]
)
print(len(l_cols_to_drop))
# df_labels_target_dynamic_raw.columns.to_list()

45


In [ ]:
%%time
(
    d_wsh_names['sub_raw']['worksheet']
    .update([df_labels_target_dynamic_raw.drop(l_cols_to_drop, axis=1).columns.values.tolist()] + 
             df_labels_target_dynamic_raw.drop(l_cols_to_drop, axis=1).fillna('').values.tolist())
)

CPU times: user 207 ms, sys: 13.1 ms, total: 220 ms
Wall time: 7.59 s


{'spreadsheetId': '1bRRcL-BBh2ngOCWS_SS9RjWr-lArOH4awCBnb80kVeA',
 'updatedCells': 272425,
 'updatedColumns': 85,
 'updatedRange': 'raw_data_per_subreddit!A1:CG3205',
 'updatedRows': 3205}

## Save: FPR target-2-target as list

Even though data isn't fully ready, want to have the output ready to make sure it's in the right format that we need.

In [ ]:
df_target_to_target_list.iloc[:5, :5]

,subreddit_name_seed,subreddit_id_seed,cluster_label,cluster_label_k,counterpart_count
0,barriesdirtysecrets,t5_52vpfg,0004-0006-0011-0013-0013,k_0063_label,8
1,winnipeghookup,t5_512v3f,0004-0006-0011-0013-0013,k_0063_label,8
2,bramptonhookups,t5_59o53r,0004-0006-0011-0013-0013,k_0063_label,8
3,torontohangoutfriends,t5_4wwf1s,0004-0006-0011-0013-0013,k_0063_label,8
4,vancouver4friends,t5_rl58z,0004-0006-0011-0013-0013,k_0063_label,8


In [ ]:
%%time

(
    d_wsh_names['clusters_t2t_fpr_raw']['worksheet']
    .update(
        [df_target_to_target_list.columns.values.tolist()] + 
        df_target_to_target_list.fillna('').values.tolist()
    )
)

CPU times: user 30.3 ms, sys: 3.32 ms, total: 33.6 ms
Wall time: 1.84 s


{'spreadsheetId': '1bRRcL-BBh2ngOCWS_SS9RjWr-lArOH4awCBnb80kVeA',
 'updatedCells': 21952,
 'updatedColumns': 7,
 'updatedRange': 'raw_clusters_fpr_ca_ca!A1:G3136',
 'updatedRows': 3136}

## We can read the data back to confirm it's as expected

In [ ]:
# Here's how to get the records as a dataframe
pd.DataFrame(
    d_wsh_names['sub_raw']['worksheet'].get_all_records()
).iloc[:5, :15]

,subreddit_id,subreddit_name,cluster_label_int,cluster_topic_mix,rated E,link_to_sub,subs_in_cluster_count,list_cluster_subreddit_names,allow_discovery,users_l7,total_users_in_subreddit_l28,rating_name,over_18,rating_short,primary_topic
0,t5_38twja,punjabiactresses,1,Mature Themes and Adult Content,TRUE,www.reddit.com/r/punjabiactresses,17,"punjabiactresses, faptopunjabiactresses, punjabitopmodels, jackieredmond, canadianbabes, maddisonnoel, irlbeauty, drama, cestvrai_, alexannlp, joyli, brooklyntrapp, saharrooo, claudiatihan, beatricebouchard, eugeniebouchard, billiebopbi...",,9338,31329,Everyone,,E,Celebrity
1,t5_46yyke,faptopunjabiactresses,1,Mature Themes and Adult Content,FALSE,www.reddit.com/r/faptopunjabiactresses,17,"punjabiactresses, faptopunjabiactresses, punjabitopmodels, jackieredmond, canadianbabes, maddisonnoel, irlbeauty, drama, cestvrai_, alexannlp, joyli, brooklyntrapp, saharrooo, claudiatihan, beatricebouchard, eugeniebouchard, billiebopbi...",,543,2258,,f,,Mature Themes and Adult Content
2,t5_4zfh9h,punjabitopmodels,1,Mature Themes and Adult Content,FALSE,www.reddit.com/r/punjabitopmodels,17,"punjabiactresses, faptopunjabiactresses, punjabitopmodels, jackieredmond, canadianbabes, maddisonnoel, irlbeauty, drama, cestvrai_, alexannlp, joyli, brooklyntrapp, saharrooo, claudiatihan, beatricebouchard, eugeniebouchard, billiebopbi...",,605,2042,,,,
3,t5_he7gg,jackieredmond,1,Mature Themes and Adult Content,TRUE,www.reddit.com/r/jackieredmond,17,"punjabiactresses, faptopunjabiactresses, punjabitopmodels, jackieredmond, canadianbabes, maddisonnoel, irlbeauty, drama, cestvrai_, alexannlp, joyli, brooklyntrapp, saharrooo, claudiatihan, beatricebouchard, eugeniebouchard, billiebopbi...",t,471,1525,Everyone,f,E,Celebrity
4,t5_29nix8,canadianbabes,1,Mature Themes and Adult Content,TRUE,www.reddit.com/r/canadianbabes,17,"punjabiactresses, faptopunjabiactresses, punjabitopmodels, jackieredmond, canadianbabes, maddisonnoel, irlbeauty, drama, cestvrai_, alexannlp, joyli, brooklyntrapp, saharrooo, claudiatihan, beatricebouchard, eugeniebouchard, billiebopbi...",,5046,17641,Everyone,,E,Mature Themes and Adult Content


# Appendix


## Additional checks on cluster depth

In [ ]:
print(df_labels_target_dynamic_raw['cluster_label'].nunique())
display(
    value_counts_and_pcts(
        df_labels_target_dynamic_raw,
        ['cluster_label'],
        top_n=10,
    )
)
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    ['cluster_label'],
    top_n=None,
    return_df=True
)['count'].describe()

388


,count,percent,cumulative_percent
cluster_label,,,
0007-0012-0020-0027-0028-0037-0038-0048-0138-0277-0409-0459-0669-0790-0951-1013-1221-1360-1464-1596-1671-1696,63,2.0%,2.0%
0007-0011-0019-0026-0027-0036-0037-0047-0136-0274-0403-0453-0658-0776-0933-0993-1200-1335-1438-1565-1638-1663,59,1.8%,3.8%
0010-0016-0029-0039-0041-0052-0055-0077-0218-0436-0628-0703-1015-1192-1435-1527-1822-2026-2189-2373-2473-2507,38,1.2%,5.0%
0010-0017-0030-0040-0042-0053-0056-0079-0223-0443-0640-0716-1032-1214-1460-1556-1856-2062-2229-2415-2519-2554,23,0.7%,5.7%
0006-0010-0018-0025-0025-0033-0034-0043,21,0.7%,6.4%
0010-0017-0030-0040-0042-0053-0056-0079-0223-0443-0639-0715-1031-1213-1458-1554-1854-2060-2227-2413-2517-2552,21,0.7%,7.0%
0006-0009-0014-0018-0018-0023-0024-0029-0087-0170-0253-0289-0431-0507-0613-0654-0801-0897-0967-1062-1111-1123,20,0.6%,7.6%
0006-0009-0014-0018-0018-0023-0024-0029-0087-0171-0254-0290-0434-0510-0617-0658-0806-0902-0973-1068-1117-1129,17,0.5%,8.2%
0009-0015-0027-0037-0039-0050-0053-0071-0201-0397-0577-0648-0937-1101-1325-1405-1678-1871-2012-2184-2276-2308,17,0.5%,8.7%


count    388.000000
mean       8.255155
std        4.862380
min        1.000000
25%        6.000000
50%        7.000000
75%        9.000000
max       63.000000
Name: count, dtype: float64

### How deep are the clusters?

Looks like some peaks around 100, 300, 1k, and 4k clusters.

In [ ]:
print(df_labels_target_dynamic_raw[col_new_cluster_name].nunique())
value_counts_and_pcts(
    df_labels_target_dynamic_raw,
    [col_new_cluster_name],
    top_n=None,
    sort_index=True,
)

20


,count,percent,cumulative_percent
cluster_label_k,,,
k_0013_label,40,1.2%,1.2%
k_0023_label,18,0.6%,1.8%
k_0041_label,39,1.2%,3.0%
k_0059_label,9,0.3%,3.3%
k_0063_label,19,0.6%,3.9%
k_0085_label,45,1.4%,5.3%
k_0118_label,267,8.3%,13.6%
k_0320_label,358,11.2%,24.8%
k_0657_label,206,6.4%,31.3%


In [ ]:
# style_df_numeric(
#     df_labels_target.tail(10),
#     # rename_cols_for_display=True,
#     l_bar_simple=[c for c in df_labels_target.columns if '_label' in c]
# )

# Filter out subs [extra]

UPDATE: For now let's include all the subreddits for QA because this list could potentially help us rate/flag subreddits that aren't rated or mis-rated and have a lot of traffic.

Now that we have even more clusters (over 3,000), it's harder to figure out where to set the threshold for clusters to exclude.

--- 

The main use case for now are SFW subs, we could save some QA time by excluding these subs:
- Exclude NSFW clusters
- Exclude place subs


~We'll use the cluster labels to discard subreddits because~
- many of the DE subreddits don't have a `primary_topic`
- if the majority of subs for a subreddits are NSFW, then we wouldn't want to recommend those anyway

In [ ]:
# # we can see that the subreddit count changes as we go 
# #  from shallow to deeper cluster counts
# value_counts_and_pcts(
#     df_labels_target['k_0118_majority_primary_topic'],
#     top_n=15,
#     reset_index=True,
#     add_col_prefix=False,
#     count_type='subreddits',
#     return_df=False,
# )

In [ ]:
# value_counts_and_pcts(
#     df_labels_target['k_3145_majority_primary_topic'],
#     top_n=15,
#     reset_index=True,
#     add_col_prefix=False,
#     count_type='subreddits',
#     return_df=False,
# )

In [ ]:
# value_counts_and_pcts(
#     df_labels_target['k_3927_majority_primary_topic'],
#     top_n=15,
#     reset_index=True,
#     add_col_prefix=False,
#     count_type='subreddits',
#     return_df=False,
# )

In [ ]:
# # And the count is slightly different from the known primary topics
# #  We still have a large number of subs w/o a primary topic
# value_counts_and_pcts(
#     df_labels_target['primary_topic'],
#     count_type='subreddits',
#     reset_index=True,
#     add_col_prefix=False,
# )

In [ ]:
print(f"{df_labels_target.shape} <- Shape before filtering")

l_manual_subs_to_remove = [
    'sexmeets1', 'fuck',
]
col_cluster_filter = 'k_3145_majority_primary_topic'
df_labels_target_clean = (
    df_labels_target[df_labels_target[col_cluster_filter] != 'Mature Themes and Adult Content']
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping NSFW clusters")

l_sensitive_topics = [
    'Military', 'Gender', 'Addiction Support',
    'Medical and Mental Health', 'Sexual Orientation',
    'Culture, Race, and Ethnicity',
]
df_labels_target_clean = (
    df_labels_target_clean[
        ~df_labels_target_clean[col_cluster_filter].isin(l_sensitive_topics)
    ]
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping Sensitive clusters")

df_labels_target_clean = (
    df_labels_target_clean[
        ~df_labels_target_clean['primary_topic'].isin(l_sensitive_topics)
    ]
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping SENSITIVE subreddits")


df_labels_target_clean = (
    df_labels_target_clean[
        ~df_labels_target_clean['subreddit_name'].isin(l_manual_subs_to_remove)
    ]
)
print(f"{df_labels_target_clean.shape} <- Shape after dropping Manual list of subreddits")

print(f"  ** TODO: instead of excluding place subs, add logic to map hierarchy **")
# df_labels_target_clean = (
#     df_labels_target_clean[df_labels_target_clean['primary_topic'] != 'Place']
# )
# print(f"{df_labels_target_clean.shape} <- Shape after dropping Place subreddits")

(3203, 74) <- Shape before filtering
(3117, 74) <- Shape after dropping NSFW clusters
(2924, 74) <- Shape after dropping Sensitive clusters
(2890, 74) <- Shape after dropping SENSITIVE subreddits
(2890, 74) <- Shape after dropping Manual list of subreddits
  ** TODO: instead of excluding place subs, add logic to map hierarchy **
